# 如何向提示中添加示例

:::info 前提条件

本指南假定您已熟悉以下内容：

- [查询分析](/docs/tutorials/rag#query-analysis)

:::

随着我们的查询分析变得越来越复杂，语言模型（LLM）可能会在某些场景下难以准确理解应该如何响应。为了提升这方面的效果，我们可以在提示中添加示例来引导语言模型。

让我们来看看如何为我们在[查询分析教程](/docs/tutorials/rag#query-analysis)中构建的LangChain YouTube视频查询分析器添加示例。

## 安装配置

### 安装依赖项

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/core zod uuid
</Npm2Yarn>
```

### 设置环境变量

```
# 可选，使用 LangSmith 以获得最佳的可观测性体验
LANGSMITH_API_KEY=your-api-key
LANGSMITH_TRACING=true

# 如果你不在无服务器环境中运行，可以减少追踪延迟
# LANGCHAIN_CALLBACKS_BACKGROUND=true
```

In [ ]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

## 查询模式

我们将定义一个查询模式，这是我们希望模型输出的内容。为了使我们的查询分析更有趣一些，我们将添加一个 `subQueries` 字段，其中包含从顶级问题中派生出的更具体的问题。

In [21]:
import { z } from "zod";

const subQueriesDescription = `
If the original question contains multiple distinct sub-questions,
or if there are more generic questions that would be helpful to answer in
order to answer the original question, write a list of all relevant sub-questions.
Make sure this list is comprehensive and covers all parts of the original question.
It's ok if there's redundancy in the sub-questions, it's better to cover all the bases than to miss some.
Make sure the sub-questions are as narrowly focused as possible in order to get the most relevant results.`

const searchSchema = z.object({
    query: z.string().describe("Primary similarity search query applied to video transcripts."),
    subQueries: z.array(z.string()).optional().describe(subQueriesDescription),
    publishYear: z.number().optional().describe("Year video was published")
})

## 查询生成

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts"
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables"

const system = `You are an expert at converting user questions into database queries.
You have access to a database of tutorial videos about a software library for building LLM-powered applications.
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them.`

const prompt = ChatPromptTemplate.fromMessages(
[
    ["system", system],
    ["placeholder", "{examples}"],
    ["human", "{question}"],
]
)
const llmWithTools = llm.withStructuredOutput(searchSchema, {
  name: "Search",
})
const queryAnalyzer = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
]);

让我们在提示中不提供任何示例的情况下尝试使用查询分析器：

In [34]:
await queryAnalyzer.invoke(
  "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

{
  query: "difference between Web Voyager and Reflection Agents",
  subQueries: [ "Do Web Voyager and Reflection Agents use LangGraph?" ]
}

## 添加示例并调整提示词

这已经相当有效，但我们可能希望它进一步分解问题，以区分关于 Web Voyager 和 Reflection Agents 的查询。

为了优化我们的查询生成结果，我们可以在提示词中添加一些输入问题的示例以及对应的标准输出查询。

In [35]:
const examples = []

In [36]:
const question = "What's chat langchain, is it a langchain template?"
const query = {
  query: "What is chat langchain and is it a langchain template?",
  subQueries: ["What is chat langchain", "What is a langchain template"],
}
examples.push({ "input": question, "toolCalls": [query] })

1

In [37]:
const question2 = "How to build multi-agent system and stream intermediate steps from it"
const query2 = {
  query: "How to build multi-agent system and stream intermediate steps from it",
  subQueries: [
    "How to build multi-agent system",
    "How to stream intermediate steps from multi-agent system",
    "How to stream intermediate steps",
  ],
}

examples.push({ "input": question2, "toolCalls": [query2] })

2

In [38]:
const question3 = "LangChain agents vs LangGraph?"
const query3 = {
  query: "What's the difference between LangChain agents and LangGraph? How do you deploy them?",
  subQueries: [
    "What are LangChain agents",
    "What is LangGraph",
    "How do you deploy LangChain agents",
    "How do you deploy LangGraph",
  ],
}
examples.push({ "input": question3, "toolCalls": [query3] });

3

现在我们需要更新提示模板和链，以便在每个提示中包含示例。由于我们正在使用LLM模型的函数调用功能，因此需要做一些额外的结构化工作，以将示例输入和输出发送给模型。我们将创建一个`toolExampleToMessages`辅助函数来为我们处理这项任务：

In [44]:
import {
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
  } from "@langchain/core/messages";
  import { v4 as uuidV4 } from "uuid";
  
  const toolExampleToMessages = (example: Record<string, any>): Array<BaseMessage> => {
    const messages: Array<BaseMessage> = [new HumanMessage({ content: example.input })];
    const openaiToolCalls = example.toolCalls.map((toolCall) => {
      return {
        id: uuidV4(),
        type: "function" as const,
        function: {
          name: "search",
          arguments: JSON.stringify(toolCall),
        },
      };
    });
  
    messages.push(new AIMessage({ content: "", additional_kwargs: { tool_calls: openaiToolCalls } }));
  
    const toolOutputs = "toolOutputs" in example ? example.toolOutputs : Array(openaiToolCalls.length).fill("You have correctly called this tool.");
    toolOutputs.forEach((output, index) => {
      messages.push(new ToolMessage({ content: output, tool_call_id: openaiToolCalls[index].id }));
    });
  
    return messages;
  }
  
  const exampleMessages = examples.map((ex) => toolExampleToMessages(ex)).flat();

In [40]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { RunnableSequence } from "@langchain/core/runnables";

const queryAnalyzerWithExamples = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
    examples: () => exampleMessages,
  },
  prompt,
  llmWithTools
]);

In [41]:
await queryAnalyzerWithExamples.invoke(
    "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

{
  query: "Difference between Web Voyager and Reflection agents, do they both use LangGraph?",
  subQueries: [
    "Difference between Web Voyager and Reflection agents",
    "Do Web Voyager and Reflection agents use LangGraph"
  ]
}

通过我们的示例，我们得到了一个稍微更细化的搜索查询。通过进一步的提示工程和优化示例，我们可以进一步提升查询生成的效果。

您可以看到，这些示例作为消息传递给了模型，详见[LangSmith 跟踪](https://smith.langchain.com/public/102829c3-69fc-4cb7-b28b-399ae2c9c008/r)。

## 下一步

现在您已经学习了一些将少量示例（few-shotting）与查询分析相结合的技术。

接下来，请查看本节中其他有关查询分析的指南，例如[如何处理高基数数据](/docs/how_to/query_high_cardinality)。